In [18]:
import json
import pprint
import re
import time
from googletrans import Translator

pp = pprint.PrettyPrinter(indent=2)

In [19]:
with open("data/famous_people_raw_final.json", "r", encoding='utf-8') as f:    # Read JSON File from data folder
    json_list = json.load(f)

In [20]:
def clean(text):

    """
    :param text: text in tamil with punctuations, numbers and alphabetic characters
    :return: cleaned text without punctuations, numbers and english characters
    """

    text = re.sub(r"\([^()]*\)", "", text) # remove text within parenthesis eg: (born in chennai)
    text = re.sub("[a-zA-Z]", "", text) # remove english characters
    text = re.sub("[0-9]", "", text) # remove numbers from the string
    text = re.sub("[!\"#$%&'()*+,\-/:;<=>?@[\]^_`{|}~]", "", text) # remove punctuations from text except fullstops
    text = re.sub(" +", " ", text) # remove extra whitespaces

    # if len(text.split())<=10:
    #     return None

    return text.replace('.', ' . ') # replace full stop by spaces

def translate_date(date):

    """
    :param date: Date as a string eg: 14 September 1997
    :return: Date converted to tamil form eg: 14 புரட்டாசி 1997
    """

    key_map = {
        'january' : "தை", 'february' : "மாசி", 'march' : "பங்குனி",
        'april' : "சித்திரை", 'may' : "வைகாசி", 'june' : "ஆணி",
        'july' : "ஆடி", 'august' : "ஆவணி", 'september' : "புரட்டாசி",
        'october' : "ஐப்பசி", 'november' : "கார்த்திகை",
        'december' : "மார்கழி"
    }

    if date == None:
        return date

    date = date.lower()
    for key in key_map:
        date = date.replace(key, key_map[key])

    return date

In [21]:
for item in json_list: # The following is manual cleaning for some data fields
    if item['பெயர்'] == 'ஏ.எல்.ராகவன்(பாடகர்)':
        item['பெயர்'] = 'ஏ.எல்.ராகவன்'
    if item['பெயர்'] == 'எஸ்.சி.கிருஷ்ணன்':
        item['பிறந்த திகதி'] = None
    if item['பெயர்'] == 'சிவமணி':
        item['பிறந்த திகதி'] = '01/12/1959'
        item['பிறந்த திகதி தமிழில்'] = '01 மார்கழி 1959'
    if item['பெயர்'] == 'ஏ. எம். ராஜா':
        item['செயற்பாட்டுக் காலம்'] = '1950-1970'
    if item['பெயர்'] == 'ஜேம்ஸ் வசந்தன்':
        item['செயற்பாட்டுக் காலம்'] = '2008-present'
    if item['பெயர்'] == 'விஷால் சந்திரசேகர்':
        item['செயற்பாட்டுக் காலம்'] = '2002–present'
    if item['பெயர்'] == 'உன்னிமேனன்':
        item['செயற்பாட்டுக் காலம்'] = '1980-present'

In [22]:
text_fields = ['அறிமுகம்', 'உள்ளடக்கம்'] # Specify text fields
date_fields = ['பிறந்த திகதி தமிழில்']
lists = ['தொழில்', 'இசை வடிவங்கள்', 'இசைக்கருவிகள்']

for item in json_list:
    for field in text_fields:
        if item[field] != None:
            item[field] = clean(item[field])
    for field in date_fields:
        item['பிறந்த மாதம்'] = None
        item[field] = translate_date(item[field])
    for field in lists:
        for i,k in enumerate(item[field]):
            item[field][i] = clean(k).strip()

    # TODO Rerun the scraper and generate data.txt and then uncomment below lines
    if 'அறிமுக ஆண்டு' in item.keys():
        del item['அறிமுக ஆண்டு']

In [23]:
from langdetect import detect
from google_trans_new import google_translator

translator = google_translator(url_suffix="in")


# translator = Translator()

def translate_birth_place(place):

    """
    :param place: place of birth in English
    :return: corresponding Thamizh translation
    """

    if place == None or place == '':
        return None

    if detect(place) == 'ta':
        return place

    place = place.lower()
    place = place.replace('british india', 'india')

    return translator.translate(place, lang_tgt='ta')


for item in json_list:

    english_text = item['பிறப்பிடம்']
    item['பிறப்பிடம் தமிழில்'] = translate_birth_place(item['பிறப்பிடம்'])

    # pp.pprint(f"{english_text} was translated as {item['பிறப்பிடம்']}")
    time.sleep(1)

In [24]:
with open('data/famous_people_cleaned_final.json', 'w', encoding='utf-8') as f:
    json.dump(json_list, f, ensure_ascii=False, indent=4)

In [27]:
with open("data/famous_people_cleaned_final.json", "r", encoding='utf-8') as f:    # Read JSON File from data folder
    json_list = json.load(f)

months = ["தை", "மாசி", "பங்குனி", "சித்திரை", "வைகாசி", "ஆணி",
    "ஆடி", "ஆவணி", "புரட்டாசி", "ஐப்பசி", "கார்த்திகை", "மார்கழி"
]
for item in json_list:
    for key in months:
        if item['பிறந்த திகதி தமிழில்'] != None:
            if key in item['பிறந்த திகதி தமிழில்']:
                item['பிறந்த மாதம்'] = key

with open('data/famous_people_cleaned_final.json', 'w', encoding='utf-8') as f:
    json.dump(json_list, f, ensure_ascii=False, indent=4)